In [1]:
from utils import read_from_jsonl
train_data = read_from_jsonl("train.jsonl")
test_data = read_from_jsonl("test.jsonl")

In [2]:
train_data[0]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'chunk_tags': [1, 1, 1, 0, 1, 1, 1, 0, 1],
 'pos_tags': [1, 4, 3, 1, 4, 4, 3, 1, 4]}

In [3]:
from copy import deepcopy
train_data_encoded = deepcopy(train_data)
test_data_encoded = deepcopy(test_data)

In [4]:
import numpy as np

curr_one_hot_encoding_mapping = {
    1: [1, 0, 0, 0],
    2: [0, 1, 0, 0],
    3: [0, 0, 1, 0],
    4: [0, 0, 0, 1]
}

prev_one_hot_encoding_mapping = {
    # 0 for start of string sequence
    0: [1, 0, 0, 0, 0],
    1: [0, 1, 0, 0, 0],
    2: [0, 0, 1, 0, 0],
    3: [0, 0, 0, 1, 0],
    4: [0, 0, 0, 0, 1]
}

for i, data in enumerate(train_data):
    for j, pos in enumerate(data["pos_tags"]):
        if j == 0:
            prev = prev_one_hot_encoding_mapping[0]
            curr = curr_one_hot_encoding_mapping[pos]
        else:
            prev = prev_one_hot_encoding_mapping[data["pos_tags"][j-1]]
            curr = curr_one_hot_encoding_mapping[pos]
        
        train_data_encoded[i]["pos_tags"][j] = np.array(prev + curr)
        

In [5]:
train_data_encoded[0]


{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'chunk_tags': [1, 1, 1, 0, 1, 1, 1, 0, 1],
 'pos_tags': [array([1, 0, 0, 0, 0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1, 0, 0, 1, 0]),
  array([0, 0, 0, 1, 0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1, 0, 0, 1, 0]),
  array([0, 0, 0, 1, 0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0, 0, 0, 0, 1])]}

In [6]:
for i in train_data_encoded:
    chunk_tags = np.array(i["chunk_tags"])
    i["chunk_tags"] = chunk_tags
    pos_tags = np.array(i["pos_tags"])
    pos_tags.reshape(1, pos_tags.shape[0], pos_tags.shape[1])
    i["pos_tags"] = pos_tags


In [7]:
train_data_encoded[0]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'chunk_tags': array([1, 1, 1, 0, 1, 1, 1, 0, 1]),
 'pos_tags': array([[1, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1]])}

In [8]:
for i in range(2):
    print(train_data_encoded[i]["pos_tags"].shape)
    print(train_data_encoded[i]["chunk_tags"].shape)

(9, 9)
(9,)
(2, 9)
(2,)


## Dataloader implementing Above things is in utils.py

In [1]:
import torch
from utils import read_from_jsonl
from utils import DataLoader
import matplotlib.pyplot as plt
train_data = read_from_jsonl("train.jsonl")
test_data = read_from_jsonl("test.jsonl")

train_data_loader = DataLoader(train_data[:2000], batch_size=len(train_data[:2000]))
test_data_loader = DataLoader(test_data, batch_size=1)

In [2]:
len(train_data), len(train_data_loader) 

(14041, 2000)

In [3]:
from utils import RecurrentPerceptron

a = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
# a.train_from_dataloader(train_data_loader, lr=0.05, nepochs=500, debug=False, print_interval=50)
a.load_model('model.pkl')

In [4]:
a.check_conditions()

(tensor(16), 16)

In [5]:
a.test_from_dataloader(test_data_loader, verbose=True)

  0%|          | 0/3453 [00:00<?, ?it/s]

tensor(0.8423)

In [9]:
# running this will overwrite the model file
# a.save_model()

File already exists, overwriting


In [6]:
# final weights
a.W, a.V

(tensor([ 21.9425,  12.3735,  10.4288,  11.9462,  15.9650, -14.3887, -10.7598,
         -14.0571, -10.5215]),
 tensor(-0.6757))